In [2]:
pip install -q langchain langchain-community chromadb sentence-transformers openai tiktoken python-dotenv langchain_openai

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install pypdf

Note: you may need to restart the kernel to use updated packages.


In [ ]:
from openai import OpenAI

# DO NOT EXPOSE THIS KEY PUBLICLY!!!!
api_key = ""
# DO NOT EXPOSE THIS KEY PUBLICLY!!!!

project_id = "proj_fHRnVJY0Oyfm1ufG1sffxa6W"

client = OpenAI(api_key=api_key, project=project_id)

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = ""

In [6]:
import pandas as pd
#df = pd.read_csv('..\Data\Student_rubric_feedback.csv', skipinitialspace=True)
df = pd.read_csv('../Data/Student_rubric_feedback.csv', skipinitialspace=True)
df.head()

,Student,Criteria,Ratings,Points,Points Awarded,Final Grade,Feedback
0,Student_1,Functionality,Code completes without errors,25,25,100,Excellent job! Your code runs without errors a...
1,Student_1,Correctness/Style,Code is stylistically correct and works as pla...,25,25,100,Excellent job! Your code runs without errors a...
2,Student_1,Analytical Detail,Analysis includes appropriate statistical test...,25,25,100,Excellent job! Your code runs without errors a...
3,Student_1,Checkpoint Completion,All checkpoints were submitted on time.,25,25,100,Excellent job! Your code runs without errors a...
4,Student_2,Functionality,Code completes without errors,25,18,67,Minor errors noted but your code runs for the ...


In [7]:
# coulmn names have trailing whitespace
df.columns = df.columns.str.strip()
expected = ["Student", "Criteria", "Ratings", "Points", "Points Awarded", "Final Grade", "Feedback"]
missing = [c for c in expected if c not in df.columns]
if missing:
  raise ValueError(f"Missing expected columns after normalization: {missing}")

# --- Clean text columns ---
for col in ["Student", "Criteria", "Ratings", "Feedback"]:
  df[col] = df[col].astype(str).str.strip()

# --- Convert numeric columns ---
df["Points"] = pd.to_numeric(df["Points"], errors="coerce").fillna(25)
df["Points Awarded"] = pd.to_numeric(df["Points Awarded"], errors="coerce").fillna(0)
df["Final Grade"] = pd.to_numeric(df["Final Grade"], errors="coerce")

# --- Add unique ID for each row ---
df["id"] = [f"row-{i}" for i in range(len(df))]

In [8]:
import os
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings

# Use raw strings or forward slashes on Windows to avoid escape issues
""""
pdf_folder = [
    "..\Data\SQL slides\Advanced SQL I copy.pdf",
    "..\Data\SQL slides\Advanced SQL II copy.pptx.pdf",
    "..\Data\Assignment_prompt.pdf",
    "..\Data\Syllabus.pdf",
]
"""
pdf_folder = [
    "../Data/SQL slides/Advanced SQL I copy.pdf",
    "../Data/SQL slides/Advanced SQL II copy.pptx.pdf",
    "../Data/Assignment_prompt.pdf",
    "../Data/Syllabus.pdf",
]

pdf_docs = {}

for file in pdf_folder:
    loader = PyPDFLoader(file)
    pages = loader.load_and_split()
    # add source metadata for each page
    for p in pages:
        p.metadata["source_name"] = file
    # store all pages for this file
    pdf_docs[file] = pages

# Flatten dictionary into one list of docs
flat_docs = [doc for pages in pdf_docs.values() for doc in pages]

embeddings = OpenAIEmbeddings()

#persist_dir = "unified_chroma_db"

In [9]:
#vectordb = Chroma.from_documents(documents=all_docs, embedding=embeddings, persist_directory="./vectordb_index")
#vectordb.persist()
# 2. Create Chroma DB from flat_docs
vectordb = Chroma.from_documents(
    documents=flat_docs,
    embedding= embeddings,
    persist_directory="./vectordb_index"
)
vectordb.persist()

/var/folders/q6/zhddk2010yzdxh2b2q3wtwdw0000gp/T/ipykernel_19149/2228153208.py:9: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectordb.persist()


In [10]:
vector_retriever = vectordb.as_retriever(search_kwargs={"k": 3})

In [11]:
for i, doc in enumerate(flat_docs[:5]):
    print(f"\n--- Raw Doc {i+1} from {doc.metadata['source_name']} ---")
    print(doc.page_content[:500])  # peek at first 500 chars


--- Raw Doc 1 from ../Data/SQL slides/Advanced SQL I copy.pdf ---
Advanced SQL I

--- Raw Doc 2 from ../Data/SQL slides/Advanced SQL I copy.pdf ---
Agenda - Schedule
1. SQL Leetcode Q
2. Window Functions
3. Break
4. COVID-19 SQL Lab
 Database systems of the past

--- Raw Doc 3 from ../Data/SQL slides/Advanced SQL I copy.pdf ---
Agenda - Announcements 
● No pre-class quiz
● TLAB #3 due 5/14
○ Early grade due date: 5/7
○ Extension due date: 5/13
● In-class end of phase project is being released THIS THURSDAY! (We 
recommend attending this review session)

--- Raw Doc 4 from ../Data/SQL slides/Advanced SQL I copy.pdf ---
Agenda - Goals
● Use SQL window functions (ROW_NUMBER, RANK, DENSE_RANK, NTILE) to 
analyze and rank data within partitions.
● Apply LAG() and LEAD() functions to compare rows across a sequence.
● Use subqueries to calculate differences between values (e.g., current vs. 
previous order amounts).

--- Raw Doc 5 from ../Data/SQL slides/Advanced SQL I copy.pdf ---
SQL Leetc

In [12]:
for file in pdf_folder:
    loader = PyPDFLoader(file)
    pages = loader.load_and_split()
    print(f"{file}: {len(pages)} pages")
    print(f"First page text: {pages[0].page_content[:500]}")

../Data/SQL slides/Advanced SQL I copy.pdf: 43 pages
First page text: Advanced SQL I
../Data/SQL slides/Advanced SQL II copy.pptx.pdf: 41 pages
First page text: Advanced SQL II
../Data/Assignment_prompt.pdf: 2 pages
First page text: Ad-hoc  Analysis  
You  are  a  data  analyst  at  an  NYC-based  wholesale  food  supplier  called  Osiris-Foods.  As  part  of  
your
 
companies
 
mission
 
to
 
make
 
more
 
data-driven
 
decisions,
 
you
 
are
 
tasked
 
with
 
generating
 
an
 
ad-hoc
 
report
 
on
 
your
 
companies
 
database.
 
You
 
will
 
report
 
on
 
descriptive
 
statistics
 
using
 
SQL
 
and
 
generate
 
respective
 
visualizations
 
using
 
pandas
 
to
 
provide
 
your
 
company
 
an
 
overview
 
of
 
its'
 
pa
../Data/Syllabus.pdf: 13 pages
First page text: Innovation  Fellowship
 Data  Science  Syllabus   Class  Summary:  The  Knowledge  House  Data  Science  Innovation  Fellowship  aims  to  provide  students  with  a  multifaceted,  
foundational
 
skill
 
set
 
essent

In [55]:
# 10–20 query samples against your retriever
queries = [
    "What is the ad-hoc analysis assignment prompt?",
    "What are the required elements of the 7-day plan (objective, 3–5 tasks, citations)?",
    "What does SQL stand for?",
    "What is ad hoc analysis?",
    "What are the grading criteria in the rubric, and how many points are allocated? Cite rows/pages.",
    "What specific slides do I need to cover based on my score?",
    "When is the most productive time to study?",
    "What counts as 'Excellent' vs 'Satisfactory' for the modeling criterion?",
    "Where do slides cover group by and having? (typos test)",
    "What feedback did Student 5 receive on their last assignment?",
    "What improvement areas were mentioned for coding style?",
    "Where are GROUP BY/HAVING explained in the slides?",
    "How can I measure whether I’ve completed my daily study goals?",
    "Are we supposed to include an ERD or schema diagram?",
    "List the deliverables required by the assignment.",
    "Based on the rubric and slides, what steps should I take to improve query optimization?",
    "Summarize common weaknesses mentioned in the instructor feedback.",
    "What are the grading criteria in the rubric and their weights?"
]

# Run each query against the retriever
for query in queries:
    print(f"\nQuery: {query}")
    docs = vector_retriever.get_relevant_documents(query)

    for i, doc in enumerate(docs, 1):
        source = doc.metadata.get("source_name", "Unknown source")
        print(f"\n--- Document {i} (from {source}) ---")
        print(doc.page_content[:500])  # preview first 500 chars




Query: What is the ad-hoc analysis assignment prompt?

--- Document 1 (from ..\Data\Assignment_prompt.pdf) ---
Ad-hoc  Analysis  
You  are  a  data  analyst  at  an  NYC-based  wholesale  food  supplier  called  Osiris-Foods.  As  part  of  
your
 
companies
 
mission
 
to
 
make
 
more
 
data-driven
 
decisions,
 
you
 
are
 
tasked
 
with
 
generating
 
an
 
ad-hoc
 
report
 
on
 
your
 
companies
 
database.
 
You
 
will
 
report
 
on
 
descriptive
 
statistics
 
using
 
SQL
 
and
 
generate
 
respective
 
visualizations
 
using
 
pandas
 
to
 
provide
 
your
 
company
 
an
 
overview
 
of
 
its'
 
pa

--- Document 2 (from ../Data/Assignment_prompt.pdf) ---
Ad-hoc  Analysis  
You  are  a  data  analyst  at  an  NYC-based  wholesale  food  supplier  called  Osiris-Foods.  As  part  of  
your
 
companies
 
mission
 
to
 
make
 
more
 
data-driven
 
decisions,
 
you
 
are
 
tasked
 
with
 
generating
 
an
 
ad-hoc
 
report
 
on
 
your
 
companies
 
database.
 
You
 
will
 
report
 
on

In [14]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model='gpt-4o-mini', temperature=0.2)

In [15]:
feedback = df.groupby('Student')["Feedback"].unique().apply(list)
feedback

Student
Student_1     [Excellent job! Your code runs without errors ...
Student_10    [Functional solution, but missed checkpoints h...
Student_2     [Minor errors noted but your code runs for the...
Student_3                   [Good try. Improvements to be made]
Student_4     [Your SQL code runs overall, but a few minor i...
Student_5     [Your submission was complete and code execute...
Student_6     [Code was incomplete however assignment was su...
Student_7                                          [Well done.]
Student_8     [You demonstrated consistency across all areas...
Student_9     [Your analysis was strong and very thorough, b...
Name: Feedback, dtype: object

In [ ]:
"""
# AI Agent: Feedback Analyzer which will do an analysis of the feedback they received.
# Check the quality of the feedback (vague, expansive).
def feedback_analyzer(feedback: list) -> list:
  #Grab context for the feedback
  docs = vector_retriever.get_relevant_documents("What grading criteria are used for SQL assignments??")
  #Create a prompt for Agent Feedback Analyzer
  prompt = f"""
""""
    You are Agent Feedback Analyzer.
    Your job is to determine the quality of the instructor feedback per student.
    The feedback is expansive if it includes specfic information.
    Example of expansive feed back is "Your SQL code runs overall, but a few minor issues should be addressed. For example, you missed some join statements.These are small fixes and don’t indicate a major bug. Queries are well-structured and stylistically strong. The analysis includes appropriate tests, metrics, and clear visuals, showing solid reasoning. Be sure to submit all checkpoints on time to stay fully on track."
    The feedback is vague of it has general information.
    Example of vague feed back: "Good try. Improvements to be made"


    Example Output:
    Student_1's {feedback} is vague
    Student_2's {feedback} is moderate
    Student_3's {feedback} is expansive.

    Output the results as a dictionary: 'Student 1: vague'
    """
"""
  # get back answer from `gpt-4o-mini` using context & prompt
  resp = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.2,
    )
  cleaned = resp.choices[0].message.content.strip()
  return cleaned
"""


In [ ]:
#fa_results = feedback_analyzer(feedback)
#fa_results

"```python\n{\n    'Student 1': 'expansive',\n    'Student 2': 'moderate',\n    'Student 3': 'vague',\n    'Student 4': 'expansive',\n    'Student 5': 'expansive',\n    'Student 6': 'vague',\n    'Student 7': 'vague',\n    'Student 8': 'expansive',\n    'Student 9': 'expansive',\n    'Student 10': 'moderate'\n}\n```"

In [18]:
#AI Agent WeakPoints
  #this AI agent shows the weakpoints or strong points the person needs to study
  #if feedback is vague -> relying on rubric rating + criteria, and assignment prompt to establish weakpoints
  #if feedback is moderate -> relying on rubric rating + criteria, assignment prompt, and instructor feedback to establish weakpoints
  #if feedback is expansive -> relying on rubric rating + criteria, assignment prompt, and more of the instructor feedback to establish weakpoints

#Process
  # Student 3's feedback was vague: "Good try. Improvements to be made" ->
  # *looks at rubric rating* "Analysis includes some visualizations or metrics but lacks depth or completeness."
  # *looks at criteria*:"Analytical Detail: If applicable, is visual analysis of the data necessary and reasonable? Fellow must demonstrate not only technical ability in generating analysis but also the ability to describe their reasoning for said analysis. For instance, do we include the necessary statistical tests, model measurements, and visual exploratory data analysis which can provide background context and insight into performance? These can include things like ANOVA, measures of central tendency, heatmaps, bar graphs, model metrics, and so on."
  # *Looks at  assignment prompt*

#Output (Return Weakpoints)
  # List of WEAKPOINTS -> "Did not include a join between sales and product categories; analysis is incomplete without category insights.-->

In [38]:
def Weakpoint_Detector(feedback) -> str:

  #Grab context for the weakpoints
  context = vector_retriever.get_relevant_documents("What is the ad-hoc analysis assignment_prompt?")

  #Create a prompt for Agent Weakpoint Detector
  prompt = f"""
  You are Agent Weakpoint Detector. Your job is to utilize the {feedback} and connect to the {context} to create weakpoints that a user will need to study
  The weakpoints have to rely on rubric rating + criteria, and assignment prompt to establish weakpoints.

 
  Return as a dictionary were each student is the key and their weakpoints are the values.
  Example: Student 1: list of weakpoints
  """
  # get back answer from `gpt-4o-mini` using context & prompt
  resp = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.2,
    )
  cleaned = resp.choices[0].message.content.strip()
  #print(cleaned)
  return cleaned



In [39]:
weakpoints = Weakpoint_Detector(feedback)
weakpoints

'Based on the feedback provided for each student and the assignment prompt, here are the identified weakpoints for each student:\n\n```python\nweakpoints = {\n    "Student_1": [\n        "Ensure to include evidence to support claims in your write-up.",\n        "Consider adding more detailed numerical analysis to strengthen your findings."\n    ],\n    "Student_2": [\n        "Review minor errors in your SQL queries to improve accuracy.",\n        "Make sure your write-up includes all pertinent numerical details."\n    ],\n    "Student_3": [\n        "Identify specific areas for improvement in your SQL queries.",\n        "Enhance your write-up with more evidence and numerical support."\n    ],\n    "Student_4": [\n        "Address minor issues in your SQL code for better performance.",\n        "Include more detailed analysis in your write-up to support your findings."\n    ],\n    "Student_5": [\n        "Ensure your code is complete and addresses all aspects of the assignment.",\n  

In [ ]:
#AI Agent: Vector Searcher:
  #This agent will look into the vector database to connect the weakpoints to the content

  #give us clusters of where the information is at (for example, Join information are in Advanced SQL I copy slides 10-12)
  #Output (Dictionary?)
    #Student 1 Weakpoint: Information Located
    #

def vector_Searcher(weakpoints: dict) -> dict:
  #This agent will look into the vector database to connect the weakpoints to the content

  #Grab context for the weakpoints
  context = vector_retriever.get_relevant_documents("What is the ad-hoc analysis assignment_prompt?")
  

  #Create a prompt for Agent Weakpoint Detector
  prompt = f""" 
  You are Agent Weakpoint Detector. You job is to utilize the {weakpoints} and {context} to give us clusters of where the information is at (for example, Join information are in Advanced SQL I copy slides 10-12)
  I want the output as a dictionary
  Example: Student 1 Weakpoint: Information Located
  
  """
  # get back answer from `gpt-4o-mini` using context & prompt
  resp = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.2,
    )
  cleaned = resp.choices[0].message.content.strip()
  #print(cleaned)
  return cleaned

In [34]:
assignment_location = vector_Searcher(weakpoints)
assignment_location

'Based on the weakpoints identified for each student and the context provided in the assignment prompt, here is a dictionary that clusters the weakpoints according to the relevant information categories:\n\n```python\nweakpoint_clusters = {\n    \'Student 1\': {\n        \'Weakpoint\': "Ensure to include pertinent numerical details to back findings.",\n        \'Information Located\': "General reporting guidelines and expectations for evidence."\n    },\n    \'Student 2\': {\n        \'Weakpoint\': "Include more specific examples of descriptive statistics used.",\n        \'Information Located\': "Descriptive statistics and their application in SQL."\n    },\n    \'Student 3\': {\n        \'Weakpoint\': "Clarify the overall structure of the report.",\n        \'Information Located\': "Report structure and organization."\n    },\n    \'Student 4\': {\n        \'Weakpoint\': "Expand on the analysis of the ERD diagram and its implications for SQL queries.",\n        \'Information Located\

In [47]:
#AI agent: Study Planner:
  #This agent will create a 7 day study plan based on weakpoints and where the information is located
def study_planner(weakpoints, assignmemt_location):

  #Could we provide our background research as context for the agent?
    context = vector_retriever.get_relevant_documents("What's on the syllabus?")

  #Create a prompt for Agent Study Planner
    prompt = f"""
    You are Agent Study Planner. Your job is to create a 7 day study plan for each student based on {weakpoints} and include where the information is located {assignmemt_location} based on the s
    A good study plan consist of SMART Goals, Spaced repetition & retrieval practice. Each day include a time block dedicated to studying that specific skill. Make a 7 day plan for each student based on their own feedback
    For example:
    Student 1: 
    Monday: Work on SQL problems. Resources = Slides 10-12
    Tuesday: Practice Join statements . Resources = Slides 5-6 Inner joins 
    """
  
   # get back answer from `gpt-4o-mini` using context & prompt
    resp = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.2,
      )
  #return resp.choices[0].message.content
  #return resp.choices[0].message.content.split()
    cleaned = resp.choices[0].message.content.replace("\\n", "\n")
    cleaned = resp.choices[0].message.content.strip()
  #print(cleaned)
    return cleaned

In [48]:
study_planner = study_planner(weakpoints,assignment_location)
study_planner

"Here's a 7-day study plan for each student based on their identified weakpoints, incorporating SMART goals, spaced repetition, and retrieval practice. Each day includes a specific focus area, resources, and a time block dedicated to studying that skill.\n\n### Student 1 Study Plan\n**Goal:** Improve write-up quality and numerical analysis.\n\n- **Monday:** \n  - Focus: Review evidence inclusion in reports.\n  - Resources: General reporting guidelines.\n  - Time Block: 2 hours.\n\n- **Tuesday:** \n  - Focus: Practice numerical analysis techniques.\n  - Resources: Slides 10-12 on data analysis.\n  - Time Block: 2 hours.\n\n- **Wednesday:** \n  - Focus: Draft a section of the write-up with evidence.\n  - Resources: Previous assignments for reference.\n  - Time Block: 1.5 hours.\n\n- **Thursday:** \n  - Focus: Peer review of write-up for evidence.\n  - Resources: Peer feedback guidelines.\n  - Time Block: 1 hour.\n\n- **Friday:** \n  - Focus: Revise write-up based on feedback.\n  - Resour

In [ ]:
#Running the functions 
weakpoints = Weakpoint_Detector(feedback)
assignment_location = vector_Searcher(weakpoints)
study_planner = study_planner(weakpoints,assignment_location)